In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization,LSTM, Reshape, SimpleRNN
from keras.layers.recurrent import GRU
import tensorflow

Using TensorFlow backend.


In [3]:
import os
import string
import cv2

In [4]:

characters = "0123456789-/"

In [5]:
all_images = ['WhatsApp Image 2020-09-15 at 01.43.31.jpeg']

In [6]:
def preprocess_data():
    
    inp_x = np.zeros((len(all_images), 480,240,1))
    inp_y = np.zeros((len(all_images), 10,len(characters)))
    for idx, image in enumerate(all_images):
        # ----------------prepare input container
        
        
        img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
        print(img.shape)
        #print(np.max(img))
        #plt.imshow(img)
        
        # ---------------------Scale images --------------
        
        img = img/255.0
        
        img = cv2.resize(img, (240,480))
        img = np.reshape(img, (480,240,1))
        
        print(img.shape)
        
        image_txt = '30-10-1999'
        
        
        
        inp_x[idx] = img

         # ------------------Define targets and code them using OneHotEncoding
        target_oht = np.zeros((10,len(characters)))
        for k, char in enumerate(image_txt):
            target_oht[k, characters.find(char)] = 1
        inp_y[idx] = target_oht
    return inp_x, inp_y

In [7]:
X, Y = preprocess_data()

(1040, 492)
(480, 240, 1)


In [8]:

X.shape

(1, 480, 240, 1)

In [9]:
train_x = X[:]
train_y = Y[:]
test_x = X[:]
test_y = Y[:]

In [14]:
model = Sequential()
def prepare_model():
    model.add(Conv2D(64, (3,3), input_shape = (480,240,1) ,padding = 'same', activation = 'tanh'))
    #model.add(Dropout(.5))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'tanh', padding = 'same',))
    #model.add(Dropout(.3))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size =(2,2),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'tanh', padding = 'same'))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (3,2),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'tanh', padding = 'same'))
  
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'tanh', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2,1),padding ='same'))
    print(model.output_shape)
    model.add(Reshape((model.output_shape[1], model.output_shape[2]*model.output_shape[3])))
    print(model.output_shape)
    #model.add(LSTM(50, batch_input_shape=(2,5,36),stateful = True, return_sequences = True))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    


    model.add(Dense(12, activation = "softmax"))
    model.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(),
              metrics=['accuracy'])
   
    return model

In [15]:

model =prepare_model()

(None, 10, 15, 32)
(None, 10, 480)


In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 480, 240, 64)      640       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 240, 120, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 240, 120, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 120, 60, 64)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 120, 60, 64)       36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 40, 30, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 40, 30, 32)       

In [17]:
hist = model.fit(train_x, train_y, epochs=50,verbose=1, validation_data=(test_x, test_y))


Epoch 1/50
1/1 [==============================] - 0s 494ms/step - loss: 2.5136 - accuracy: 0.0000e+00 - val_loss: 2.4631 - val_accuracy: 0.1000
Epoch 2/50
1/1 [==============================] - 0s 32ms/step - loss: 2.4631 - accuracy: 0.1000 - val_loss: 2.5679 - val_accuracy: 0.1000
Epoch 3/50
1/1 [==============================] - 0s 33ms/step - loss: 2.5679 - accuracy: 0.1000 - val_loss: 2.2443 - val_accuracy: 0.1000
Epoch 4/50
1/1 [==============================] - 0s 33ms/step - loss: 2.2443 - accuracy: 0.1000 - val_loss: 2.1770 - val_accuracy: 0.2000
Epoch 5/50
1/1 [==============================] - 0s 33ms/step - loss: 2.1770 - accuracy: 0.2000 - val_loss: 2.0798 - val_accuracy: 0.3000
Epoch 6/50
1/1 [==============================] - 0s 32ms/step - loss: 2.0798 - accuracy: 0.3000 - val_loss: 1.9748 - val_accuracy: 0.3000
Epoch 7/50
1/1 [==============================] - 0s 35ms/step - loss: 1.9748 - accuracy: 0.3000 - val_loss: 1.8733 - val_accuracy: 0.5000
Epoch 8/50
1/1 [======